<a href="https://colab.research.google.com/github/marquinamaria/UCB_MDSv4_ML/blob/main/ARL_FP_Growth_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Publicacion que propuso el Algoritmo

- http://myweb.sabanciuniv.edu/rdehkharghani/files/2016/02/The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-Morgan-Kaufmann-2011.pdf

- A Guided FP-growth algorithm for multitude-targeted mining of big data. arXiv:1803.06632 [cs.DB].
https://doi.org/10.48550/arXiv.1803.06632



# Breve Explicacion de como funciona

El algoritmo FP-growth (Frequent Pattern growth) es una técnica eficiente y escalable de minería de datos que permite el descubrimiento patrones frecuentes en conjuntos de datos transaccionales o de cestas de la compra sin generar conjuntos de elementos candidatos. Fue propuesto por Jiawei Han y Jian Pei en 2000 como una mejora del algoritmo Apriori, que es otra técnica comúnmente utilizada para la minería de patrones frecuentes.

Los siguientes son los pasos para el algoritmo de crecimiento de FP

1. Escanear la base de datos una vez para encontrar un conjunto frecuente de 1 elemento **1-itemset** (patrón de elemento único)
2. Ordenar los elementos frecuentes en orden descendente de frecuencia, **f-list**
3. Escanear la base de datos nuevamente, y construir el árbol FP, **FP-tree**
4. Construir el árbol FP condicional en la secuencia de orden inverso de **f-list** y luego generar un conjunto de elementos frecuentes


# Tipos de Entrada

Algunos de los tipos de datos de entrada típicos para FP-Growth son los siguientes:

1. **Conjunto de datos transaccionales**: FP-Growth se utiliza comúnmente para analizar conjuntos de datos que contienen registros de transacciones. Cada transacción es una lista de elementos o artículos. Los elementos pueden representar productos comprados por un cliente, palabras en documentos, elementos de interacción en un sitio web o cualquier otro tipo de transacción donde se registren elementos que ocurren juntos.

2. **Conjunto de datos de cestas de la compra**: FP-Growth también se utiliza en análisis de cestas de la compra en tiendas minoristas. En este caso, cada transacción representa una cesta de la compra de un cliente, y los elementos son los productos comprados.

3. **Conjunto de datos de interacciones en línea**: En análisis de registros de usuarios en un sitio web o aplicación, las transacciones pueden representar las acciones realizadas por un usuario, como hacer clic en enlaces o ver páginas. Los elementos pueden representar páginas web, categorías de productos, palabras clave de búsqueda, etc.

# Casos de Uso

Aquí detallamos algunos casos de uso comunes de FP-Growth:

1. **Análisis de cestas de la compra en comercio minorista**: FP-Growth se usa para identificar patrones de compra frecuentes en los registros de ventas de una tienda minorista. Esto puede ayudar a la tienda a tomar decisiones sobre cómo organizar sus productos en las estanterías, cómo hacer recomendaciones a los clientes o cómo planificar promociones basadas en los productos que se compran juntos con frecuencia.

2. **Recomendación de productos**: En sitios de comercio electrónico y aplicaciones de recomendación, FP-Growth se puede utilizar para analizar el historial de compras de los usuarios y sugerir productos relacionados en función de patrones frecuentes. Esto mejora la experiencia del usuario y puede aumentar las ventas cruzadas.

3. **Detección de fraudes en tarjetas de crédito**: FP-Growth se utiliza para analizar patrones de uso de tarjetas de crédito y detectar actividades fraudulentas. Por ejemplo, si ciertos patrones de gastos o compras inusuales se detectan con frecuencia en las transacciones de tarjetas de crédito, el algoritmo podría alertar sobre posibles fraudes.

4. **Análisis de clics en publicidad en línea**: En la publicidad en línea, FP-Growth puede utilizarse para analizar patrones de clics en anuncios y determinar qué anuncios o categorías de anuncios suelen atraer a los usuarios juntos. Esto puede ayudar a las empresas a optimizar sus estrategias publicitarias.

5. **Análisis de registros de usuarios en sitios web y aplicaciones**: FP-Growth se utiliza para analizar los registros de actividad de los usuarios en sitios web y aplicaciones para identificar patrones de interacción comunes. Esto puede ayudar a mejorar la usabilidad del sitio web, personalizar la experiencia del usuario y aumentar la retención de usuarios.

6. **Detección de enfermedades en registros médicos**: En la atención médica, FP-Growth se utiliza para analizar registros médicos y encontrar patrones de síntomas o diagnósticos que ocurren juntos con frecuencia. Esto puede ayudar en la detección temprana de enfermedades o en la identificación de factores de riesgo.

7. **Segmentación de clientes**: FP-Growth se utiliza en la segmentación de clientes para identificar grupos de clientes con comportamientos de compra similares. Esto permite a las empresas adaptar sus estrategias de marketing y servicios a grupos específicos de clientes.

8. **Análisis de texto**: FP-Growth también se puede aplicar a la minería de texto para descubrir patrones de palabras o términos que aparecen juntos con frecuencia en documentos. Esto es útil en la clasificación de documentos, resumen de texto y análisis de sentimientos.

# Source Code